The Omega Manufacturing Company has discontinued the production of a certain unprofitable product line. This act created considerable excess production capacity. Management is considering devoting this excess capacity to one or more of three products; call them products $1$, $2$, and $3$. The available capacity on the machines that might limit output is summarized in the following table:

| Machine Type | Available Time (Machine hours per week) |
| :--- | :---: |
| Miling machine | 500 |
| Lathe | 350 |
| Grinder | 150 |

The number of machine hours required for each unit of the respective products is

| Machine Type | Product 1 | Product 2 | Product 3 |
| :--- | :---: | :---: | :---: |
| Milling machine | 9 | 3 | 5 |
| Lathe | 5 | 4 | 0 |
| Grinder | 3 | 0 | 2 |


The sales department indicates that the sales potential for products $1$ and $2$ exceeds the maximum production rate and that the sales potential for product $3$ is $20$ units per week. The unit profit would be $\$50$, $\$20$, and $\$25$, respectively, on products $1$, $2$, and $3$. The objective is to determine how much of each product Omega should produce to maximize profit.


In [16]:
from pyomo.environ import *

In [17]:
model = AbstractModel()

In [18]:
model.products = Set()
model.machines = Set()

In [19]:
model.available_time = Param(model.machines)
model.machine_hour_per_product = Param(model.machines, model.products)
model.unit_profit = Param(model.products)

In [20]:
model.quotas = Var(model.products, within=NonNegativeReals)

In [21]:
def total_profit(model):
    return summation(model.unit_profit, model.quotas)


model.profit = Objective(rule=total_profit, sense=maximize)

In [22]:
def maximum_machine_hours(model, machine):
    return sum(model.machine_hour_per_product[machine, product] * model.quotas[product] for product in model.products) <= model.available_time[machine]

model.machines_constraint = Constraint(model.machines, rule=maximum_machine_hours)    

In [23]:
def sales_potential(model, product):
    if product == 3:
        return model.quotas[product] <= 20
    else:
        return Constraint.Feasible

model.sales_potential_constraint = Constraint(model.products, rule=sales_potential)

In [24]:
data = { None: { 'products': { None: (1, 2, 3 ) },
                 'machines': { None: ('Milling machine', 'Lathe', 'Grinder')} ,
                 'available_time': { 'Milling machine': 500,
                                     'Lathe': 350,
                                     'Grinder': 150
                                   },
                 'machine_hour_per_product': { ('Milling machine', 1): 9,
                                               ('Milling machine', 2): 3,
                                               ('Milling machine', 3): 5,
                                               ('Lathe', 1): 5,
                                               ('Lathe', 2): 4,
                                               ('Lathe', 3): 0,
                                               ('Grinder', 1): 3,
                                               ('Grinder', 2): 0,
                                               ('Grinder', 3): 2
                                             },
                 'unit_profit': { 1: 50,
                                  2: 20,
                                  3: 25
                                }
               } 
       }

In [25]:
instance = model.create_instance(data)
instance.pprint()

ERROR: Constructing component 'sales_potential_constraint' from data=None
    failed:
        ValueError: Invalid constraint expression. The constraint expression
        resolved to a trivial Boolean (True) instead of a Pyomo object. Please
        modify your rule to return Constraint.Feasible instead of True.

    Error thrown for Constraint 'sales_potential_constraint[1]'


ValueError: Invalid constraint expression. The constraint expression resolved to a trivial Boolean (True) instead of a Pyomo object. Please modify your rule to return Constraint.Feasible instead of True.

Error thrown for Constraint 'sales_potential_constraint[1]'

In [0]:
opt = SolverFactory('glpk')

In [0]:
results = opt.solve(instance)
instance.solutions.store_to(results)
print(results)